In [ ]:
%conf 
numRows=5
showLog=true

In [ ]:
%env 
ETL_CONF_DATA_URL=s3a://dask-data/nyc-taxi/2015
ETL_CONF_JOB_URL=https://raw.githubusercontent.com/tripl-ai/arc-starter/master/examples/kubernetes

In [ ]:
{
  "type": "DelimitedExtract",
  "name": "extract data from tripdata public data repository",
  "environments": ["production", "test"],
  "inputURI": ${ETL_CONF_DATA_URL}"/*tripdata_2015-08.csv",
  "outputView": "tripdata0_raw",            
  "delimiter": "Comma",
  "quote" : "DoubleQuote",
  "header": true,
  "persist": true
}

In [ ]:
{
  "type": "TypingTransform",
  "name": "apply tripdata schema 0 data types",
  "environments": ["production", "test"],
  "schemaURI": ${ETL_CONF_JOB_URL}"/green_tripdata0.json",
  "inputView": "tripdata0_raw",            
  "outputView": "tripdata0"
}

In [ ]:
%sqlvalidate name="ensure no errors exist after data typing" environments=production,test
SELECT
  SUM(error) = 0 AS valid
  ,TO_JSON(
      NAMED_STRUCT(
        'count', COUNT(error), 
        'errors', SUM(error)
      )
  ) AS message
FROM (
  SELECT 
    CASE 
      WHEN SIZE(_errors) > 0 THEN 1 
      ELSE 0 
    END AS error 
  FROM tripdata0
) input_table

In [ ]:
%sql name="ensure a query can be executed" environments=production,test persist=true outputView=trip_filtered
SELECT * 
FROM tripdata0
WHERE store_and_fwd_flag = TRUE